In [53]:
import torch
from torch import nn

In [54]:
char_box = ['e', 'h', 'l', 'o']
char_hello = [1, 0, 2, 2, 3]
char_ohlol = [3, 1, 2, 3, 2]

In [55]:
inputs = torch.tensor(char_hello).view(5,1) # input size = 5
targets = torch.tensor(char_ohlol) # for CrossEntropy

#### inputs (5,1) 因为序列长度为5, batch为1, 即RNN序列的每一个单元，都有batch_size个输入同时进行输入。input_size为4因为在inputs中有四类字母
#### embedding后 embedding (5,1,10) 即对于RNN序列的每一个单元我都有一个dim=10的向量作为输入。即embedding_size = 10
#### 经过一个hidden后，hidden (5,1,4), hidden_size = 4，同样是因为输出的character种类有四种，经过softmax层会输出四种character的概率，所以hidden_size是4

In [56]:
class MyNet(nn.Module):
    def __init__(self, input_size, embedding_size, hidden_size, num_layers):
        super(MyNet, self).__init__()
        self.emb = nn.Embedding(num_embeddings=input_size, embedding_dim=embedding_size)
        self.rnn = nn.RNN(input_size=embedding_size, hidden_size=hidden_size, num_layers=num_layers)
        self.lstm = nn.LSTM(input_size=embedding_size, hidden_size=hidden_size, num_layers=num_layers)
        self.gru = nn.GRU(input_size=embedding_size, hidden_size=hidden_size, num_layers=num_layers)

    def forward(self, inputs): # 输入的inputs是一个(5,1), RNN中的inputs应该是(seq_len, batch_size, input_size)
        inputs = self.emb(inputs)
        
        # # RNN hn的形状(num_layers, batch_size, hidden_size)
        # hn = torch.zeros(num_layers, inputs.size(1), hidden_size)
        # outputs, hn = self.rnn(inputs, hn) # outputs的形状应该是(seq_len, batch_size, hidden_size)

        # # LSTM 输出有两个hn和cn，初始化和前向传播的时候要注意
        # hn = torch.zeros(num_layers, inputs.size(1), hidden_size) # LSTM hn的形状与 RNN相同(num_layers, batch_size, hidden_size)
        # cn = torch.zeros(num_layers, inputs.size(1), hidden_size) # LSTM cn的形状与其对应的hn的形状相同，因为要进行对应位置元素的相乘(num_layers, batch_size, hidden_size)
        # outputs, (hn, cn) = self.lstm(inputs, (hn, cn)) 

        # GRU 门控循环单元，使用方法与RNN完全相同
        hn = torch.zeros(num_layers, inputs.size(1), hidden_size)
        outputs, hn = self.gru(inputs, hn)

        return outputs.view(-1,4) #(5,1,4)->(5,4)

input_size = 4 # 4类character 'e', 'h', 'l', 'o'
embedding_size = 10
hidden_size = 4 # 4类character 'e', 'h', 'l', 'o'
num_layers = 1

mynet = MyNet(input_size=input_size, embedding_size=embedding_size, hidden_size=hidden_size, num_layers=num_layers)
mynet.forward(inputs)


tensor([[ 0.3562,  0.0273,  0.7150,  0.3079],
        [ 0.3312,  0.6649,  0.4634, -0.4636],
        [ 0.4077,  0.0918, -0.6117, -0.3159],
        [ 0.4445, -0.1631, -0.8916, -0.2115],
        [-0.1126,  0.1363, -0.6604,  0.1144]], grad_fn=<ViewBackward0>)

In [57]:
epoch = 30
loss_fn = nn.CrossEntropyLoss()
optim = torch.optim.Adam(mynet.parameters(), lr=0.05)

if __name__ == "__main__":
    for i in range(epoch):
        print(f'-----------第{i+1}轮训练开始-----------')
        mynet.train()

        outputs = mynet(inputs)
        result = outputs.argmax(axis=1) # argmax()只能处理二维矩阵
        for num in result:
            print(char_box[num], end="") #输出结果
        print() # 换行
        
        loss = loss_fn(outputs, targets) # outputs的形状必须是(N,C), target的形状是(N,)的一维数组。N代表seq_len, C代表用于分类的种类
        optim.zero_grad()
        loss.backward()
        optim.step()

        print(f'第{i+1}轮训练结束，loss={loss}')


-----------第1轮训练开始-----------
lheeh
第1轮训练结束，loss=1.5876580476760864
-----------第2轮训练开始-----------
lheeo
第2轮训练结束，loss=1.2646535634994507
-----------第3轮训练开始-----------
lhool
第3轮训练结束，loss=1.0389230251312256
-----------第4轮训练开始-----------
olool
第4轮训练结束，loss=0.8855615854263306
-----------第5轮训练开始-----------
ollol
第5轮训练结束，loss=0.7852229475975037
-----------第6轮训练开始-----------
ollol
第6轮训练结束，loss=0.7265177369117737
-----------第7轮训练开始-----------
ollol
第7轮训练结束，loss=0.6921847462654114
-----------第8轮训练开始-----------
ohlol
第8轮训练结束，loss=0.6697393655776978
-----------第9轮训练开始-----------
ohlol
第9轮训练结束，loss=0.6537488698959351
-----------第10轮训练开始-----------
ohlol
第10轮训练结束，loss=0.6418073177337646
-----------第11轮训练开始-----------
ohlol
第11轮训练结束，loss=0.6322439908981323
-----------第12轮训练开始-----------
ohlol
第12轮训练结束，loss=0.623809814453125
-----------第13轮训练开始-----------
ohlol
第13轮训练结束，loss=0.6157842874526978
-----------第14轮训练开始-----------
ohlol
第14轮训练结束，loss=0.6078770160675049
-----------第15轮训练开始-----------
ohlol
第1